# Monte Carlo Tennis Simulations

There are two complementary approaches to computing the probability of winning a game, tiebreak, set, or match in tennis.

**Path enumeration**  
All possible score paths are enumerated, and the probability of winning is obtained by summing over these paths. This approach relies on closed-form expressions for deuce and tiebreak scenarios.

**Monte Carlo simulation**  
A large number of games, tiebreaks, sets, or matches are simulated by randomly sampling each point outcome according to the players’ probabilities of winning a point on serve. The win probability is then estimated as the fraction of simulated outcomes won.

In Notebooks 1–4, we focused on the path enumeration approach:
- [1 - Game Win Probability](1%20-%20Game%20Win%20Probability.ipynb)
- [2 - Tiebreak Win Probability](2%20-%20Tiebreak%20Win%20Probability.ipynb)
- [3 - Set Win Probability](3%20-%20Set%20Win%20Probability.ipynb)
- [4 - Match Win Probability](4%20-%20Match%20Win%20Probability.ipynb)

In this notebook, we turn to the second approach. We perform Monte Carlo simulations and compare the resulting probability estimates with those obtained via path enumeration, validating that both methods produce consistent results.

## Setup and Imports

In [ ]:
from matplotlib import pyplot as plt
from itertools  import cycle
import numpy as np
import os, sys, time

# add path to the 'tennis_lab' package if not in PYTHONPATH already 
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from tennis_lab.core.game_score     import GameScore
from tennis_lab.core.match_format   import MatchFormat
from tennis_lab.core.match_score    import MatchScore
from tennis_lab.core.set_score      import SetScore
from tennis_lab.core.tiebreak_score import TiebreakScore

from tennis_lab.montecarlo.game_simulation     import probabilityServerWinsGame as probabilityServerWinsGame_MC
from tennis_lab.montecarlo.tiebreak_simulation import probabilityP1WinsTiebreak as probabilityP1WinsTiebreak_MC
from tennis_lab.montecarlo.set_simulation      import probabilityP1WinsSet      as probabilityP1WinsSet_MC
from tennis_lab.montecarlo.match_simulation    import probabilityP1WinsMatch    as probabilityP1WinsMatch_MC

from tennis_lab.paths.game_probability     import loadCachedFunction as loadCachedFunction_Game
from tennis_lab.paths.tiebreak_probability import loadCachedFunction as loadCachedFunction_Tiebreak
from tennis_lab.paths.set_probability      import probabilityP1WinsSet
from tennis_lab.paths.match_probability    import probabilityP1WinsMatch

## Game Simulation

This section validates game level probability calculations. We simulate thousands of games and compare the fraction of games won by the server with the corresponding theoretical (cached) values.

The Monte Carlo function `probabilityServerWinsGame_MC` simulates games point by point, randomly determining the winner of each point based on the probability of winning a point on serve. The theoretical values are obtained from path enumeration using a closed form solution for deuce scenarios.

In [ ]:
# MONTE-CARLO SIMULATION: GAME
# Calculate the probability that the player serving wins the game using MC simulation.
# Compare results with the cached values previously calculated using the path enumeration method.
t0 = time.time()
    
# Calculate the win probability from multiple initial scores.
INIT_SCORES    = [GameScore(2,0), GameScore(0,0), GameScore(0,2)] 
PLAYER_SERVING = 1

# Probability that the player serving wins the point.
Ps = np.linspace(0, 1, 30)

# Carry out MC simulation
N = 10000     # how many games to simulate for each probability 'p'
Ys = [[probabilityServerWinsGame_MC(initScore, PLAYER_SERVING, p, N) for p in Ps] for initScore in INIT_SCORES]

# Load cached data previously calculated using the path enumeration method.
Ts = []
for score in INIT_SCORES:
    cachedFunc = loadCachedFunction_Game(score, PLAYER_SERVING)        
    Ts.append([cachedFunc(p) for p in Ps])

# Display results
plt.xlabel("probability of winning the point on serve")
plt.ylabel("probability of winning the game on serve")
plt.title ("Probability of Winning the Game on Serve - MC Simulation")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)

colors1 = cycle(["lightblue", "lightgrey", "lightgreen"])
colors2 = cycle(["blue", "grey", "green"])
for i, ys in enumerate(Ys):
    label = f"simulation,     initScore={INIT_SCORES[i].asPoints(1)}"
    plt.scatter(Ps, ys, s=70, facecolors=next(colors1), edgecolors=next(colors2), label=label, alpha=0.7)

colors1 = cycle(["blue", "grey", "green"])
for i, ts in enumerate(Ts):
    label = f"path method, initScore={INIT_SCORES[i].asPoints(1)}"
    plt.plot(Ps, ts, linewidth=0.75, color=next(colors1), label=label)

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

## Tiebreak Simulation
We consider two complementary scenarios.

**Scenario 1** fixes the starting score at 0–0 and varies Player 2’s probability of winning a point on serve. This isolates the impact of the opponent’s serve strength on the overall tiebreak win probability.

**Scenario 2** fixes Player 2’s serve probability and varies the starting score within the tiebreak. This allows us to study how the current score and the implied serving order influence the probability of winning from different points in the tiebreak.

In [ ]:
# MONTE-CARLO SIMULATION: TIEBREAKS - Scenarios 1
# Calculate the probability that Player1 wins the tiebreak by MC simulation.
# Compare results with the cached values previously calculated using the path enumeration method.
# Start at 0-0 and simulates for multiple probabilities that Player2 wins the point on serve.
t0 = time.time()

IS_SUPER       = False
INIT_SCORE     = TiebreakScore(0, 0, IS_SUPER)
PLAYER_SERVING = 1            # which player is serving next point
P1s = np.linspace(0, 1, 20)   # probability that Player1 wins the point when serving (x-axis)
P2s = [0.15, 0.50, 0.85]      # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 4000   # how many tiebreaks to simulate for each probability pair (P1, P2)
Ys = {p2: [probabilityP1WinsTiebreak_MC(INIT_SCORE, PLAYER_SERVING, p1, p2, N) for p1 in P1s] for p2 in P2s}

# Load cached data previously calculated using the path enumeration method.
cachedFunc = loadCachedFunction_Tiebreak(INIT_SCORE, PLAYER_SERVING)
Ts = {p2: [cachedFunc(p1, p2) for p1 in P1s] for p2 in P2s}

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the tiebreak")
plt.title ("Probability of Player1 Winning the Tiebreak - MC Simulation", fontsize=10)
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles  = cycle(['-', '--', ':'])
markers = cycle(['o', 's' , 'p']) 

for p2 in P2s:   # plot a family of curves
    plt.scatter(P1s, Ys[p2], s=60, facecolors='lightgrey', marker=next(markers), edgecolors='black', alpha=0.7, label=f"simulation,    p2={p2}")
    plt.plot   (P1s, Ts[p2], linewidth=0.8, color='black', linestyle=next(styles), label=f"path mehod, p2={p2}")

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

In [ ]:
# MONTE-CARLO SIMULATION: TIEBREAKS - Scenario 2
# Calculate the probability that Player1 wins the tiebreak by MC simulation.
# Compare results with the cached values previously calculated using the path enumeration method.
# Fix the probability that Player2 wins a point when serving and carry out the simulation from
# multiple initial scores. 
t0 = time.time()

IS_SUPER    = False
INIT_SCORES = [TiebreakScore(2, 0, IS_SUPER), TiebreakScore(0, 5, IS_SUPER),
               TiebreakScore(3, 3, IS_SUPER), TiebreakScore(4, 6, IS_SUPER)]
PLAYER_SERVING = 2            # which player is serving next point
P1s = np.linspace(0, 1, 20)   # probability that Player1 wins the point when serving (x-axis)
P2  = 0.5                     # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 4000   # how many tiebreaks to simulate for each probability pair (P1, P2)
Ys = {score: [probabilityP1WinsTiebreak_MC(score, PLAYER_SERVING, p1, P2, N) for p1 in P1s] for score in INIT_SCORES}

# Load cached data previously calculated by adding up score scenario probabilities
Ts = {}
for score in INIT_SCORES:
    cachedFunc = loadCachedFunction_Tiebreak(score, PLAYER_SERVING)
    Ts[score] = [cachedFunc(p1, P2) for p1 in P1s]

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the tiebreak")
plt.title ("Probability of Player1 Winning the Tiebreak - MC Simulation")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles = cycle(['-', '--', '-.', ':'])
markers = cycle(['o', 's' , 'p', 'd'])

for score in INIT_SCORES:   # plot a family of curves
    plt.scatter(P1s, Ys[score], s=50, facecolors='lightgrey', marker=next(markers), edgecolors='black', alpha=0.7, label=f"simulation,    init_score={score}")
    plt.plot   (P1s, Ts[score], linewidth=0.8, color='black', linestyle=next(styles), label=f"path mehod, init_score={score}")

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

## Set Simulation

Set simulations are computationally more expensive since each set contains multiple games (and potentially a tiebreak).<br>
The Monte Carlo approach simulates every point within every game of the set.

**Scenario 1** starts from 0-0 and varies the opponent's serve-winning probability.  
**Scenario 2** explores different starting scores within the set (e.g., mid-game scores like 30-0).

In [ ]:
# MONTE-CARLO SIMULATION: SET - Part 1
# Calculate the probability that Player1 wins the set by MC simulation.
# The granularity of the MC simulation is 'point' (not 'game').
# Compare results with the cached values previously calculated using the path enumeration method.
# Start at 0-0 and simulates for multiple probabilities that Player2 wins the point on serve.
t0 = time.time()

INIT_SCORE     = SetScore(0, 0, isFinalSet=False)
PLAYER_SERVING = 1            # which player is serving next point
P1s = np.linspace(0, 1, 20)   # probability that Player1 wins the point when serving (x-axis)
P2s = [0.25, 0.50, 0.75]      # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 1000    # how many sets to simulate for each probability pair (P1, P2)
Ys = {}
for p2 in P2s:
    Ys[p2] = [probabilityP1WinsSet_MC(INIT_SCORE, PLAYER_SERVING, p1, p2, N) for p1 in P1s]

# Calculate theoretical values (note: probabilityP1WinsSet expects an iterable of P1 values)
Ts = {}
for p2 in P2s:
    Ts[p2] = probabilityP1WinsSet(INIT_SCORE, PLAYER_SERVING, P1s, p2)

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the set")
plt.title ("Probability of Player1 Winning the Set - MC Simulation")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles  = cycle(['-', '--', ':'])
markers = cycle(['o', 's' , 'p'])

for idx, p2 in enumerate(P2s):   # plot a family of curves
    plt.scatter(P1s, Ys[p2], s=50, facecolors='lightgrey', marker=next(markers), edgecolors='black', alpha=0.7, label=f"simulation,    p2={p2}")
    plt.plot   (P1s, Ts[p2], linewidth=0.8, color='black', linestyle=next(styles), label=f"path mehod, p2={p2}")

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

In [ ]:
# MONTE-CARLO SIMULATION: SETS - Part 2
# Calculate the probability that Player1 wins the set by MC simulation.
# The granularity of the MC simulation is 'point' (not 'game').
# Compare results with the cached values previously calculated using the path enumeration method.
# Fix the probability that Player2 wins a point when serving and carry out the simulation from
# multiple initial scores. 
t0 = time.time()

INIT_SCORES = [SetScore(0, 0, False, MatchFormat(), GameScore(0, 0)), 
               SetScore(0, 0, False, MatchFormat(), GameScore(2, 0))]
PLAYER_SERVING = 1            # which player is serving next point
P1s = np.linspace(0, 1, 20)   # probability that Player1 wins the point when serving (x-axis)
P2  = 0.5                     # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 1000    # how many sets to simulate for each probability pair (P1, P2)
Ys = {}
for score in INIT_SCORES:
    Ys[score] = [probabilityP1WinsSet_MC(score, PLAYER_SERVING, p1, P2, N) for p1 in P1s]

# Calculate theoretical values (note: probabilityP1WinsSet expects an iterable of P1 values)
Ts = {}
for score in INIT_SCORES:
    Ts[score] = probabilityP1WinsSet(score, PLAYER_SERVING, P1s, P2)

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the set")
plt.title (f"Probability of Player1 Winning the Set - MC Simulation (p2={P2})")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles  = cycle(['-', '--', '-.', ':'])
markers = cycle(['o', 's' , 'p', 'd'])

for score in INIT_SCORES:   # plot a family of curves
    plt.scatter(P1s, Ys[score], s=40, facecolors='lightgrey', marker=next(markers), edgecolors='black', alpha=0.7, label=f"simulation,    init_score={score}")
    plt.plot   (P1s, Ts[score], linewidth=0.8, color='black', linestyle=next(styles), label=f"path mehod, init_score={score}")

plt.legend(fontsize=7)
print(f"Simulation time: {time.time() - t0:.2f}s")

## Match Simulation

Full match simulations are the most computationally intensive, as they simulate every point of every game of every set.<br> 
A best-of-3 match typically contains 150-250 points; a best-of-5 match can have 300+ points.

**Scenario 1** simulates best-of-3 matches starting from 0-0 with varying opponent serve strength.  
**Scenario 2** simulates best-of-5 matches from various set scores (e.g., 1-0, 2-0, 0-2, 2-2).

In [ ]:
# MONTE-CARLO SIMULATION: MATCH - Scenario 1
# Calculate the probability that Player1 wins the match by MC simulation.
# The granularity of the MC simulation is 'point' (not 'game' or 'set').
# Compare results with the cached values previously calculated using the path enumeration method.
# Start at 0-0 and simulates for multiple probabilities that Player2 wins the point on serve.
t0 = time.time()

BEST_OF        = 3
INIT_SCORE     = MatchScore(0, 0, MatchFormat(bestOfSets=BEST_OF))
PLAYER_SERVING = 1            # which player is serving next point
P1s = np.linspace(0, 1, 30)   # probability that Player1 wins the point when serving (x-axis)
P2s = [0.30, 0.50, 0.7]       # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 500    # how many matches to simulate for each probability pair (P1, P2)
Ys = {}
for p2 in P2s:
    Ys[p2] = [probabilityP1WinsMatch_MC(INIT_SCORE, PLAYER_SERVING, p1, p2, N) for p1 in P1s]

# Calculate theoretical values (note: probabilityP1WinsMatch expects an iterable of P1 values)
Ts = {}
for p2 in P2s:
    Ts[p2] = probabilityP1WinsMatch(INIT_SCORE, PLAYER_SERVING, P1s, p2)

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the match")
plt.title ("Probability of Player1 Winning the Match - MC Simulation")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles  = cycle(['-', '--', ':'])
markers = cycle(['o', 's' , 'p'])

for idx, p2 in enumerate(P2s):   # plot a family of curves
    plt.scatter(P1s, Ys[p2], s=40, facecolors='lightgrey', marker=next(markers), edgecolors='black', alpha=0.7,  label=f"simulation,    p2={p2}")
    plt.plot   (P1s, Ts[p2], linewidth=0.8, color='black', linestyle=next(styles), label=f"path method, p2={p2}")

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

In [ ]:
# MONTE-CARLO SIMULATION: MATCH - Scenario 2
# Calculate the probability that Player1 wins the match by MC simulation.
# The granularity of the MC simulation is 'point' (not 'game' or 'set').
# Compare results with the cached values previously calculated using the path enumeration method.
# Fix the probability that Player2 wins a point when serving and carry out the simulation from
# multiple initial scores. 
t0 = time.time()

BEST_OF     = 5
MF          = MatchFormat(bestOfSets=BEST_OF)
INIT_SCORES = [MatchScore(1, 0, MF), MatchScore(2, 0, MF),
               MatchScore(0, 2, MF), MatchScore(2, 2, MF)]
PLAYER_SERVING = 1            # which player is serving next point
P1s = np.linspace(0, 1, 30)   # probability that Player1 wins the point when serving (x-axis)
P2  = 0.50                    # probability that Player2 wins the point when serving (parameter)

# Carry out MC simulation
N = 500    # how many matches to simulate for each probability pair (P1, P2)
Ys = {}
for score in INIT_SCORES:
    Ys[score] = [probabilityP1WinsMatch_MC(score, PLAYER_SERVING, p1, P2, N) for p1 in P1s]

# Calculate theoretical values (note: probabilityP1WinsMatch expects an iterable of P1 values)
Ts = {}
for score in INIT_SCORES:
    Ts[score] = probabilityP1WinsMatch(score, PLAYER_SERVING, P1s, P2)

# Display results
plt.xlabel("probability of winning the point on serve for Player1")
plt.ylabel("probability of winning the match")
plt.title (f"Probability of Player1 Winning the Match - MC Simulation (p2={P2})")
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(linewidth=0.2)
styles  = cycle(['-', '--', '-.', ':'])
markers = cycle(['o', 's' , 'p', 'd'])

for score in INIT_SCORES:   # plot a family of curves
    sets1, sets2 = score.sets(1)
    label = f"simulation init score = {sets1}-{sets2}, 0-0, 0-0"
    plt.scatter(P1s, Ys[score], s=40, facecolors='lightgrey', edgecolors='black', alpha=0.7)
    plt.plot   (P1s, Ts[score], linewidth=0.8, color='black', linestyle=next(styles), label=label)

plt.legend(fontsize=6)
print(f"Simulation time: {time.time() - t0:.2f}s")

## Match Length Analysis

How does match length (in number of points) depend on the players' serve-winning probabilities?

We simulate many matches across a grid of (P1, P2) values and measure the average number of
points played. Matches between evenly-matched players tend to be longer, while mismatches
end quickly.

In [ ]:
# Simulate matches and calculate average match length
import random
from tennis_lab.core.match import Match

t0 = time.time()

# Define match format
BEST_OF = 3
matchFormat = MatchFormat(bestOfSets=BEST_OF)

# Define grid of serve probabilities
P1s = np.linspace(0.25, 0.75, 50)
P2s = np.linspace(0.25, 0.75, 50)

# Numnber of matches to simulate per (P1, P2) pair
N = 100  

def simulateMatchLength(matchFormat, p1, p2, numSims):
    """Simulate matches and return mean number of points played."""
    lengths = []
    for _ in range(numSims):
        match = Match(playerServing=1, matchFormat=matchFormat)
        while not match.isOver:
            server = match.servesNext
            prob = p1 if server == 1 else p2
            winner = server if random.random() < prob else (3 - server)
            match.recordPoint(winner)
        lengths.append(len(match.pointHistory))
    return np.mean(lengths)

# Build the heatmap data
matchLengths = np.zeros((len(P2s), len(P1s)))
for i, p2 in enumerate(P2s):
    for j, p1 in enumerate(P1s):
        print(f"\rCalculating: p1={p1:.2f},p2={p2:.2f}...", end='', flush=1)
        matchLengths[i, j] = simulateMatchLength(matchFormat, p1, p2, N)
print(f"\rSimulation time: {time.time() - t0:.2f}s              ")

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(matchLengths, origin='lower', aspect='auto',
               extent=[P1s[0], P1s[-1], P2s[0], P2s[-1]], cmap='viridis')
plt.colorbar(im, label='Average points per match')
ax.set_xlabel("P1 serve win probability", fontsize=10)
ax.set_ylabel("P2 serve win probability", fontsize=10)
ax.set_title(f"Average Match Length (Best of {BEST_OF})", fontsize=12)
plt.tight_layout()
plt.show()

### Match Length Distribution

For a specific pair of serve probabilities (P1, P2), we can visualize the full distribution 
of match lengths rather than just the mean. This reveals the variability in match duration.

In [ ]:
# Simulate match lengths for a specific (P1, P2) and plot the distribution
# Uses joblib for parallel execution with batched jobs
from joblib import Parallel, delayed
from scipy.stats import gaussian_kde
import os
t0 = time.time()

BEST_OF = 5
matchFormat = MatchFormat(bestOfSets=BEST_OF)

# Serve probabilities
P1 = 0.9
P2 = 0.9

N = 100000  # number of matches to simulate

def simulateBatch(matchFormat, p1, p2, numSims):
    """Simulate a batch of matches and return their lengths in points."""
    lengths = []
    for _ in range(numSims):
        match = Match(playerServing=1, matchFormat=matchFormat)
        while not match.isOver:
            server = match.servesNext
            prob = p1 if server == 1 else p2
            winner = server if random.random() < prob else (3 - server)
            match.recordPoint(winner)
        lengths.append(len(match.pointHistory))
    return lengths

# Partition work across available CPU cores
n_workers = os.cpu_count()
batch_size = N // n_workers
remainder = N % n_workers

# Create batch sizes (distribute remainder across first few workers)
batch_sizes = [batch_size + (1 if i < remainder else 0) for i in range(n_workers)]

# Run simulations in parallel (one task per CPU core)
results = Parallel(n_jobs=-1, verbose=1)(
    delayed(simulateBatch)(matchFormat, P1, P2, bs) for bs in batch_sizes
)

# Flatten results
lengths = np.array([length for batch in results for length in batch])

# Compute kernel density estimate
kde = gaussian_kde(lengths)
x = np.linspace(lengths.min(), lengths.max(), 200)
density = kde(x)

# Plot density
fig, ax = plt.subplots(figsize=(7, 5))

ax.fill_between(x, density, alpha=0.4, color='steelblue')
ax.plot(x, density, color='steelblue', linewidth=1.5)
ax.axvline(np.mean(lengths), color='red', linestyle='--', linewidth=1.5, label=f'Mean = {np.mean(lengths):.0f}')

ax.set_xlabel("Match Length (points)", fontsize=10)
ax.set_ylabel("Probability Density", fontsize=10)
ax.set_title(f"Distribution of Match Length (Best of {BEST_OF}, P1={P1}, P2={P2})", fontsize=11)
ax.legend(fontsize=9)
ax.grid(alpha=0.3, linewidth=0.5)
plt.tight_layout()
plt.show()

print(f"Simulation time: {time.time() - t0:.2f}s")
print(f"Workers: {n_workers}, Batch size: ~{batch_size}")
print(f"Mean: {np.mean(lengths):.1f} points")
print(f"Std:  {np.std(lengths):.1f} points")
print(f"Min:  {np.min(lengths)} points")
print(f"Max:  {np.max(lengths)} points")

### Deuce Analysis

A deuce occurs when the score is tied at 40-40 in a game, or at 6-6 (and beyond) in a tiebreak. We count how many deuces occur per match as a function of serve-winning probability.

When both players have equal serve probabilities, we expect more deuces at higher probabilities since stronger servers hold serve more reliably, leading to closer games that reach deuce more often.

In [ ]:
# Simulate matches and count deuces as a function of serve probability
# Uses joblib for parallel execution with batched jobs
from joblib import Parallel, delayed
from scipy.stats import mode
import os
t0 = time.time()

BEST_OF = 3
matchFormat = MatchFormat(bestOfSets=BEST_OF)

# Serve probabilities (P1 = P2 = P)
Ps = np.linspace(0.5, 0.9, 15)

N = 10000  # number of matches per probability value

def countDeuces(pointHistory):
    """
    Count deuce situations from point history.
    Returns (game_deuces, tiebreak_deuces).
    """
    game_deuces = 0
    tiebreak_deuces = 0
    gameP1, gameP2 = 0, 0
    inTiebreak = False
    gamesP1, gamesP2 = 0, 0
    
    for winner in pointHistory:
        if winner == 1:
            gameP1 += 1
        else:
            gameP2 += 1
        
        if inTiebreak:
            if gameP1 >= 6 and gameP2 >= 6 and gameP1 == gameP2:
                tiebreak_deuces += 1
            if (gameP1 >= 7 or gameP2 >= 7) and abs(gameP1 - gameP2) >= 2:
                inTiebreak = False
                gamesP1, gamesP2 = 0, 0
                gameP1, gameP2 = 0, 0
        else:
            if gameP1 >= 3 and gameP2 >= 3 and gameP1 == gameP2:
                game_deuces += 1
            if (gameP1 >= 4 or gameP2 >= 4) and abs(gameP1 - gameP2) >= 2:
                if gameP1 > gameP2:
                    gamesP1 += 1
                else:
                    gamesP2 += 1
                gameP1, gameP2 = 0, 0
                if gamesP1 == 6 and gamesP2 == 6:
                    inTiebreak = True
                elif (gamesP1 >= 6 or gamesP2 >= 6) and abs(gamesP1 - gamesP2) >= 2:
                    gamesP1, gamesP2 = 0, 0
    
    return game_deuces, tiebreak_deuces

def simulateBatchDeuces(matchFormat, p1, p2, numSims):
    """Simulate a batch of matches and return deuce counts."""
    results = []
    for _ in range(numSims):
        match = Match(playerServing=1, matchFormat=matchFormat)
        while not match.isOver:
            server = match.servesNext
            prob = p1 if server == 1 else p2
            winner = server if random.random() < prob else (3 - server)
            match.recordPoint(winner)
        game_d, tb_d = countDeuces(match.pointHistory)
        results.append((game_d, tb_d))
    return results

# Partition work across available CPU cores
n_workers = os.cpu_count()

# Store results for each probability
means_game = []
means_tiebreak = []
means_total = []
modes_game = []
modes_tiebreak = []
modes_total = []

for p in Ps:
    print(f"\rSimulating P={p:.2f}...", end='', flush=True)
    
    batch_size = N // n_workers
    remainder = N % n_workers
    batch_sizes = [batch_size + (1 if i < remainder else 0) for i in range(n_workers)]
    
    results = Parallel(n_jobs=-1)(
        delayed(simulateBatchDeuces)(matchFormat, p, p, bs) for bs in batch_sizes
    )
    
    all_results = [r for batch in results for r in batch]
    game_deuces = np.array([r[0] for r in all_results])
    tiebreak_deuces = np.array([r[1] for r in all_results])
    total_deuces = game_deuces + tiebreak_deuces
    
    means_game.append(np.mean(game_deuces))
    means_tiebreak.append(np.mean(tiebreak_deuces))
    means_total.append(np.mean(total_deuces))
    modes_game.append(mode(game_deuces, keepdims=False).mode)
    modes_tiebreak.append(mode(tiebreak_deuces, keepdims=False).mode)
    modes_total.append(mode(total_deuces, keepdims=False).mode)

print(f"\rSimulation time: {time.time() - t0:.2f}s              ")

# Plot mean and mode side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean plot
ax = axes[0]
ax.plot(Ps, means_game    , 'o-', color='lightblue'  , markeredgecolor='black', linewidth=1.0, markersize=10, label='Game Deuces')
ax.plot(Ps, means_tiebreak, 's-', color='lightblue'  , markeredgecolor='black', linewidth=1.0, markersize= 9, label='Tiebreak Deuces')
ax.plot(Ps, means_total   , 'o-', color='forestgreen', markeredgecolor='black', linewidth=1.0, markersize=10, label='Total Deuces')
ax.set_xlabel("Serve Win Probability (P1 = P2)", fontsize=10)
ax.set_ylabel("Mean Deuces per Match", fontsize=10)
ax.set_title(f"Mean Deuces vs Serve Probability (Best of {BEST_OF})", fontsize=12)
ax.set_facecolor((0.97, 0.97, 0.97))
ax.legend(fontsize=9)
ax.grid(alpha=0.3, linewidth=0.5)

# Mode plot
ax = axes[1]
ax.plot(Ps, modes_game    , 'o-', color='lightblue'  , markeredgecolor='black', linewidth=1.0, markersize=10, label='Game Deuces')
ax.plot(Ps, modes_tiebreak, 's-', color='lightblue'  , markeredgecolor='black', linewidth=1.0, markersize= 9, label='Tiebreak Deuces')
ax.plot(Ps, modes_total   , 'o-', color='forestgreen', markeredgecolor='black', linewidth=1.0, markersize=12, label='Total Deuces')
ax.set_xlabel("Serve Win Probability (P1 = P2)", fontsize=10)
ax.set_ylabel("Mode Deuces per Match", fontsize=10)
ax.set_title(f"Mode Deuces vs Serve Probability (Best of {BEST_OF})", fontsize=12)
ax.set_facecolor((0.97, 0.97, 0.97))
ax.legend(fontsize=9)
ax.grid(alpha=0.3, linewidth=0.5)

plt.tight_layout()
plt.show()